In [271]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from skimage import measure
import statistics
import glob

In [292]:
def RGB(array):
    return array[:,:,0], array[:,:,1], array[:,:,2]

def NCC(R,G,B):
    I = R + G + B
   
    I[I == 0] = 1000000
    
    r = R / I
    g = G / I
    b = 1 - r - g

    return r, g ,b 

def find_max_mode(list1):
    list_table = statistics._counts(list1)
    len_table = len(list_table)

    if len_table == 1:
        max_mode = statistics.mode(list1)
    else:
        new_list = []
        for i in range(len_table):
            new_list.append(list_table[i][0])
        max_mode = max(new_list) # use the max value here
    return max_mode

def most_element(liste):
    numeral=[[liste.count(nb), nb] for nb in liste]
    numeral.sort(key=lambda x:x[0], reverse=True)
    return(numeral[0][1])

In [293]:
filenames = sorted(glob.glob('Orange/*'))
filenames

['Orange/orange01.jpg',
 'Orange/orange02.jpg',
 'Orange/orange03.jpg',
 'Orange/orange04.jpg',
 'Orange/orange05.jpg',
 'Orange/orange06.jpg',
 'Orange/orange07.jpg',
 'Orange/orange08.jpg',
 'Orange/orange09.jpg',
 'Orange/orange10.png',
 'Orange/orange11.png',
 'Orange/orange12.jpg',
 'Orange/orange13.jpg',
 'Orange/orange14.jpg',
 'Orange/orange15.jpg',
 'Orange/orange16.jpg',
 'Orange/orange17.jpg',
 'Orange/orange18.jpg',
 'Orange/orange19.jpg',
 'Orange/orange20.png']

In [294]:
def create_binarized_image(file):
    img = cv2.imread(file)
    
#     plt.figure(figsize=(18,6))
#     plt.subplot(131)
#     plt.imshow(plt.imread(file))
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    im_th = thresh.copy()
    im_floodfill = im_th.copy()
    
    height, width = im_th.shape[:2]
    mask = np.zeros((height+2, width+2), np.uint8)
    
    cv2.floodFill(im_floodfill, mask, (0,0), 255)
    
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    
    im_out = im_th | im_floodfill_inv
    
    thresh = im_out.copy()
    ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
    
    if file[0:6] == 'Orange': thresh = cv2.dilate(thresh, ellipse, iterations = 50)
    else: thresh = cv2.dilate(thresh, ellipse, iterations = 1)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, ellipse, iterations = 1)
    
    #find all your connected components (white blobs in your image)
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)
    #connectedComponentswithStats yields every seperated component with information on each of them, such as size
    #the following part is just taking out the background which is also considered a component, but most of the time we don't want that.
    sizes = stats[1:, -1]; nb_components = nb_components - 1

    # minimum size of particles we want to keep (number of pixels)
    #here, it's a fixed value, but you can set it as you want, eg the mean of the sizes or whatever
    min_size = 200

    #your answer image
    thresh2 = np.zeros((output.shape))
    #for every component in the image, you keep it only if it's above min_size
    for i in range(0, nb_components):
        if sizes[i] >= min_size:
            thresh2[output == i + 1] = 255
    
    data_new = cv2.imread(file,0)
    v_max = 200
    v_min = 50
    
    data_new[(data_new <= v_max) & (data_new >= v_min)] = 1
    data_new[(data_new >= v_max) & (data_new <= v_min)] = 0
    
    
    labels= measure.label(thresh2)
    #labels= measure.label(data_new)
    #props = measure.regionprops(labels, data_new)
    props = measure.regionprops(labels, thresh2)
    eccentricity_list = []
    for props in props: eccentricity_list.append(props.eccentricity)
    eccentricity_list = np.array(eccentricity_list)
    print('eccentricity:', eccentricity_list.min())
    #print('eccentricity:', props[0].eccentricity)
    
    image = plt.imread(file)
    thresh2[thresh2 == 255] = 1

    R_img,G_img,B_img = RGB(image)

    new_image_R = R_img * thresh2
    new_image_G = G_img * thresh2
    new_image_B = B_img * thresh2

    new_img = image.copy()
    new_img[:,:,0] = new_image_R
    new_img[:,:,1] = new_image_G
    new_img[:,:,2] = new_image_B
    
#     plt.figure(figsize=[18,6])
#     plt.subplot(131)
    #plt.imshow(new_img)
    #plt.colorbar()
    
    
    r,g,b = NCC(new_image_R, new_image_G, new_image_B)
    b = 1-r-g
    b[b < 0] = 0

    colors = list(zip(r.flatten(), g.flatten(),b.flatten()))
    
    from statistics import mode
    new_r = filter(lambda a: a != 0, r.flatten())
    new_g = filter(lambda a: a != 0, g.flatten())
    
    new_r = filter(lambda a: a != 0.3333333333333333, new_r)
    new_g = filter(lambda a: a != 0.3333333333333333, new_g)

    #X = mode(new_r)
    #Y = mode(new_g)
    try:
        X = mode(new_r)
    except Exception:
        try:
            X = most_element(list(new_r))
        except:
            r_last_resort = filter(lambda a: a != 0, r.flatten())
            X = np.mean(list(r_last_resort))
    
    try:
        Y = mode(new_g)
    except Exception:
        try:
            Y = most_element(list(new_g))
        except:
            g_last_resort = filter(lambda a: a != 0, g.flatten())
            Y = np.mean(list(g_last_resort))
    
    #Y = most_element(list(new_g))
    
    #X = find_max_mode(new_r)
    #Y = find_max_mode(new_g)
    #if type(Y) == np.float64: print('r,g values:', X, Y)
    #if type(Y) != np.float64: print('r,g values:', X, Y[0])
    
    print('r,g values:', X, Y)

#     plt.subplot(132)
#     x = np.linspace(0,1,100)
#     plt.scatter(r.flatten(), g.flatten(), s = 2, alpha=0.3, c = colors)
#     plt.plot(x, 1-x, color='black')
#     plt.axvline(x = X, color='red')
#     plt.axhline(y = Y, color='green')
#     plt.xlim(0,1)
#     plt.ylim(0,1)
    
    new = np.zeros([10,10,3])
    new[:,:,0] = X
    new[:,:,1] = Y
    new[:,:,2] = 1-X-Y
    new = (new*500).astype(int)
    new[new>255] = 255
    
#     plt.subplot(133)
#     plt.imshow(new)
#     plt.show()

    return eccentricity_list.min(), X, Y

In [295]:
labels = []
orange_data = []
for file in filenames:
    print(file)
    
    eccentricity, r, g = create_binarized_image(file)
    
    if '.png' or '.jpg' in file:
        #labels.append(file[14:-4])
        orange_data.append({'label':file[13:-4], 'eccentricity': eccentricity, 'r': r, 'g': g})
    if '.jpeg' in file:
        #labels.append(file[14:-5])
        orange_data.append({'label':file[13:-5], 'eccentricity': eccentricity, 'r': r, 'g': g})
        
    print()

Orange/orange01.jpg
eccentricity: 0.3967140622713511
r,g values: 0.6666666666666666 0.3157894736842105

Orange/orange02.jpg
eccentricity: 0.38082052459545696
r,g values: 0.6538461538461539 0.3324607329842932

Orange/orange03.jpg
eccentricity: 0.5071748589685998
r,g values: 0.673469387755102 0.33421052631578946

Orange/orange04.jpg
eccentricity: 0.1341645106779717
r,g values: 0.6666666666666666 0.3346456692913386

Orange/orange05.jpg
eccentricity: 0.29080378758440006
r,g values: 0.6781914893617021 0.32180851063829785

Orange/orange06.jpg
eccentricity: 0.46689998057059257
r,g values: 0.75 0.25

Orange/orange07.jpg
eccentricity: 0.33707355410513706
r,g values: 0.6666666666666666 0.3125

Orange/orange08.jpg
eccentricity: 0.27654601997318057
r,g values: 0.6666666666666666 0.25

Orange/orange09.jpg
eccentricity: 0.5512626076242068
r,g values: 0.7323529411764705 0.25588235294117645

Orange/orange10.png
eccentricity: 0.24262446327862083
r,g values: 0.5801526627512943 0.3613231641961136

Orange

In [296]:
orange_df = pd.DataFrame(orange_data)
orange_df.to_csv('orange.csv', header=True, index=False)